In [1]:
import sys
sys.path.append("..")

from src import uilc
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
s = 60
W = 0.2
H = 0.05

# Plot value
xline = np.linspace(-W/2, W/2 , 400)
yline = np.linspace(-W/2, W/2 , 400)
X, Y = np.meshgrid(xline, yline)

In [ ]:
esc_n = uilc.esc.get_nmax(s, W, H)
esc_d = uilc.esc.coefficient(s ,esc_n) * H
esc_arr = uilc.utils.uniformarray(esc_d, esc_n) 
esc_arr_s = uilc.utils.get_2d_array(esc_arr, esc_arr, dim=2)

In [ ]:
esc_intensity = uilc.utils.lambertian_distribution(xline, np.array([[[0]]]), esc_arr, s, H)
esc_y = esc_intensity[0][0]
esc_y = esc_y/esc_y.max()
plt.plot(xline, esc_y, label="intensity of esc method")
plt.axhline(y=esc_y.mean(), color='b', label="Mean Value:{}".format(esc_y.mean()))
plt.ylim(0, 1.2)
plt.legend()
plt.show()